In [1]:
import winsound as ws
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import cv2
from sklearn.cluster import KMeans
from pathlib import Path

In [2]:
dataBooks = pd.read_csv("dataset\\books.csv")
#dataBooks = dataBooks[9700:10000]
dataBooks

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
5,6,11870085,11870085,16827462,226,525478817,9.780525e+12,John Green,2012.0,The Fault in Our Stars,...,2346404,2478609,140739,47994,92723,327550,698471,1311871,https://images.gr-assets.com/books/1360206420m...,https://images.gr-assets.com/books/1360206420s...
6,7,5907,5907,1540236,969,618260307,9.780618e+12,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,...,2071616,2196809,37653,46023,76784,288649,665635,1119718,https://images.gr-assets.com/books/1372847500m...,https://images.gr-assets.com/books/1372847500s...
7,8,5107,5107,3036731,360,316769177,9.780317e+12,J.D. Salinger,1951.0,The Catcher in the Rye,...,2044241,2120637,44920,109383,185520,455042,661516,709176,https://images.gr-assets.com/books/1398034300m...,https://images.gr-assets.com/books/1398034300s...
8,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,Angels & Demons,...,2001311,2078754,25112,77841,145740,458429,716569,680175,https://images.gr-assets.com/books/1303390735m...,https://images.gr-assets.com/books/1303390735s...
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,...,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...


In [3]:
class Bookeeper:
    
    def __init__(self, isbn10, bookid):
        self.bookid = bookid
        self.isbn10 = str(isbn10).zfill(10)
        self.urlbook = "https://www.amazon.com/gp/search/ref=sr_adv_b/?search-alias=stripbooks&unfiltered=1&field-keywords=&field-author=&field-title=&field-isbn="+self.isbn10+"&field-publisher=&node=&field-p_n_condition-type=&p_n_feature_browse-bin=&field-age_range=&field-language=&field-dateop=During&field-datemod=&field-dateyear=&sort=relevanceexprank"
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
        self.page = requests.get(self.urlbook, headers=headers)
        self.bs = BeautifulSoup( self.page.text, "html.parser")
            
    #get the list price
    def getListPrice(self):
        
        try:
            costo1 = float(self.bs.findAll("span", attrs = {"class": "a-size-base-plus a-color-secondary a-text-strike"})[0].text.replace("$", ""))
        except:
            costo1 = -1
        try:
            costo2 = float(self.bs.findAll("span", attrs = {"class": "a-offscreen"})[0].text.replace("$", ""))
        except:
            costo2 = -1
        try:
            costo3 = float(self.bs.findAll("span", attrs = {"class": "a-size-base a-color-base"})[0].text.replace("$", ""))
        except:
            costo3 = -1
            
        self.cost = max([costo1, costo2, costo3])
        
        if( self.cost < 0):
            raise ValueError('Cannot keep cost of the book')
            
        return self.cost
    
    
    #gets the COVER images
    def saveImageCover(self, location = "D:\\foto\\sampleImages\\", url = None):
        if(url == None):
            #gets source
            self.sourceCover = self.bs.findAll("img", attrs={"class" : "s-access-image cfMarker"})[0]["src"]
        else:
            self.sourceCover = url
        #gets image data
        self.img_data = requests.get(self.sourceCover).content
        with open(location+str(self.bookid)+".jpg", 'wb') as handler:
            handler.write(self.img_data)

In [4]:
class CoverColorPredict:
    #setting up palette
    palette = np.array([[0,0,0], [255,255,255], [255,0,0], [0, 255, 0], [0,0,255], [255,255,0], [255, 0, 255], [0, 255, 255], [153, 51, 0], [255,102,0]], np.float64)
    palettelabels = ["Black", "White", "Red", "Green", "Blue", "Yellow", "Violet", "Cyan", "Brown", "Orange", ];
    
    
    #use id_book like img_name
    def __init__(self, img_name):
        # load the image and convert it from BGR to RGB so that
        # we can dispaly it with matplotlib
        self.img_name = img_name
        self.image = cv2.imread('D:\\foto\\sampleImages\\'+img_name+'.jpg')
        self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB) 
        # reshape the image to be a list of pixels
        self.image = self.image.reshape((self.image.shape[0] * self.image.shape[1], 3))
        # cluster the pixel intensities
        self.clt = KMeans(n_clusters = len(CoverColorPredict.palette), init=CoverColorPredict.palette, n_init=1)
        self.clt.fit(self.image)
        
        
    #returns index of the palette and the name of color predicte
    def predictColor(self):
        #summurize data of the image
        self.labels = []
        self.results = []
        self.r = []
        self.g = []
        self.b = []
        for a in range(len(CoverColorPredict.palette)):
            self.labels.append(a)
            self.results.append(0)
            self.r.append(int(self.clt.cluster_centers_[a][0]))
            self.g.append(int(self.clt.cluster_centers_[a][1]))
            self.b.append(int(self.clt.cluster_centers_[a][2]))

        for i in self.clt.labels_:
            self.results[i] = self.results[i]+1
        #create dataset to summarize data
        self.colorresults = pd.DataFrame(data = { "labels" : self.labels, "n_pixels": self.results, "value_r" : self.r, "value_g": self.g, "value_b": self.b})
        self.maxcolor = self.colorresults["n_pixels"].idxmax()
        self.maincolor = self.clt.cluster_centers_[self.maxcolor].astype(int)
        self.predictedcolor = self.clt.predict([self.maincolor])
        #returns palette id and the label
        return self.predictedcolor, CoverColorPredict.palettelabels[self.predictedcolor[0]]
        
    
    

In [5]:
pricelist = []
colorlist = []
coloridlist = []

for index, row in dataBooks.iterrows():
    print("Elbarating id: "+str(row["id"])+" bookid: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
    
    #3 times
    maxattemps = 1
    for i in range(maxattemps):
        if(i != 0):
            print("Retry id: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
            
        try:

            bk = Bookeeper(row["isbn"], row["book_id"])
            bk.saveImageCover(url = str(row["image_url"]))
            price = bk.getListPrice()
            pricelist.append(price)
            break;
        except:
            print("Error in bookKeeper id: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
            if(i == maxattemps-1):
                pricelist.append(None)
    
    try:
        
        col = CoverColorPredict(str(row["book_id"]))
        colorpredicted = col.predictColor()
        colorlist.append(colorpredicted[1])
        coloridlist.append(colorpredicted[0][0])
    except:
        print("Error in color predict id: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
        colorlist.append(None)
        coloridlist.append(None)
    
print("DONE")
ws.Beep(500,500)
dataBooks["price(USD)"] = pricelist
dataBooks["color_name"] = colorlist
dataBooks["color_id"] = coloridlist


Elbarating id: 9701 bookid: 25067046 isbn10 1484724984
Elbarating id: 9702 bookid: 128048 isbn10 618250743
Elbarating id: 9703 bookid: 22474626 isbn10 nan
Error in bookKeeper id: 22474626 isbn10 nan
Elbarating id: 9704 bookid: 95660 isbn10 316115002
Elbarating id: 9705 bookid: 17794657 isbn10 nan
Error in bookKeeper id: 17794657 isbn10 nan
Elbarating id: 9706 bookid: 18243828 isbn10 nan
Error in bookKeeper id: 18243828 isbn10 nan
Elbarating id: 9707 bookid: 171201 isbn10 571198775
Elbarating id: 9708 bookid: 261161 isbn10 316115045
Elbarating id: 9709 bookid: 13031445 isbn10 1451668201
Elbarating id: 9710 bookid: 397867 isbn10 333761189
Elbarating id: 9711 bookid: 104088 isbn10 451458265
Elbarating id: 9712 bookid: 17333174 isbn10 425270696
Elbarating id: 9713 bookid: 15761241 isbn10 nan
Error in bookKeeper id: 15761241 isbn10 nan
Elbarating id: 9714 bookid: 23196 isbn10 553344862
Elbarating id: 9715 bookid: 701738 isbn10 1400096685
Elbarating id: 9716 bookid: 586516 isbn10 425182134
E

Elbarating id: 9842 bookid: 24019187 isbn10 1250058902
Elbarating id: 9843 bookid: 71867 isbn10 439858054
Elbarating id: 9844 bookid: 17235347 isbn10 nan
Error in bookKeeper id: 17235347 isbn10 nan
Elbarating id: 9845 bookid: 13360101 isbn10 345533232
Elbarating id: 9846 bookid: 1099097 isbn10 6640613
Elbarating id: 9847 bookid: 20685373 isbn10 307720659
Elbarating id: 9848 bookid: 61899 isbn10 671578855
Elbarating id: 9849 bookid: 36638 isbn10 385327900
Elbarating id: 9850 bookid: 17563539 isbn10 nan
Error in bookKeeper id: 17563539 isbn10 nan
Elbarating id: 9851 bookid: 784222 isbn10 1421513242
Elbarating id: 9852 bookid: 31266 isbn10 752858653
Elbarating id: 9853 bookid: 690955 isbn10 1596430699
Elbarating id: 9854 bookid: 15398 isbn10 743225740
Elbarating id: 9855 bookid: 5266667 isbn10 60874198
Elbarating id: 9856 bookid: 268955 isbn10 679824243
Elbarating id: 9857 bookid: 10461 isbn10 1573225053
Elbarating id: 9858 bookid: 383196 isbn10 9649007903
Elbarating id: 9859 bookid: 2677

Elbarating id: 9982 bookid: 12444298 isbn10 385344422
Elbarating id: 9983 bookid: 8539798 isbn10 2849659266
Elbarating id: 9984 bookid: 764165 isbn10 140255087
Elbarating id: 9985 bookid: 19688 isbn10 425176932
Elbarating id: 9986 bookid: 183092 isbn10 310257689
Elbarating id: 9987 bookid: 8087038 isbn10 312651198
Elbarating id: 9988 bookid: 129237 isbn10 674017722
Elbarating id: 9989 bookid: 13489518 isbn10 nan
Error in bookKeeper id: 13489518 isbn10 nan
Elbarating id: 9990 bookid: 294081 isbn10 140143912
Elbarating id: 9991 bookid: 101094 isbn10 385425139
Elbarating id: 9992 bookid: 13616278 isbn10 575113294
Elbarating id: 9993 bookid: 4936457 isbn10 224079948
Elbarating id: 9994 bookid: 4769651 isbn10 810983559
Elbarating id: 9995 bookid: 15613 isbn10 1416523723
Elbarating id: 9996 bookid: 7130616 isbn10 441019455
Elbarating id: 9997 bookid: 208324 isbn10 067973371X
Elbarating id: 9998 bookid: 77431 isbn10 039330762X
Elbarating id: 9999 bookid: 8565083 isbn10 61711527
Elbarating id:

In [ ]:
dataBooks[:550]

In [ ]:
dataBooks = dataBooks[0:550]

In [6]:
dataBooks.to_csv("dataset\\new\\books25.csv")

In [3]:
#Check all files
listalldata = []
for i in range(1,25):
    data = pd.read_csv("dataset\\new\\books"+str(i)+".csv")
    listalldata.append(data)
alldata = pd.concat(listalldata, ignore_index=True)
alldata = alldata.drop(axis = 1, columns=["Unnamed: 0", "color_id"])
#drop double books
axes = alldata["book_id"].value_counts().axes[0].tolist()
values = alldata["book_id"].value_counts().values.tolist()

for i in range(len(values)):
    if(values[i] > 1):
        indexes = alldata[alldata.book_id == axes[i]].index.tolist()
        alldata = alldata.drop(indexes[1:])
alldata[alldata.book_id ==930]

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,price(USD),color_name
32,33,930,929,1558965,220,739326228,9.780739e+12,Arthur Golden,1997.0,Memoirs of a Geisha,...,25605,23500,59033,258700,517157,559782,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,2.99,White


In [66]:
#not found books images 
whitebookid = alldata[alldata.color_name == "White"].dropna(axis=0, subset=["isbn"])["book_id"].values.tolist()
listwhite = []
for whiteid in whitebookid:
    if(not Path("D:\\foto\\sampleImages\\"+str(whiteid)+".jpg").is_file()):
        listwhite.append(whiteid)

print("White to download: "+str(len(listwhite)))
        
for bookid in listwhite[:500]:
    try:
        isbn = alldata[alldata.book_id == bookid]["isbn"].values[0]
        print("Elaborating "+str(bookid)+" whith isbn "+str(isbn))
        bk = Bookeeper(alldata[alldata.book_id == bookid]["isbn"].values[0], bookid)
        bk.saveImageCover()
        img = cv2.imread("D:\\foto\\sampleImages\\"+str(bookid)+".jpg")
        crop_img = img[0:218, 45:173]
        cv2.imwrite("D:\\foto\\sampleImages\\"+str(bookid)+".jpg", crop_img)
    except:
        print("Error in bookKeeper id: "+str(bookid))
        if(i == maxattemps-1):
            pricelist.append(None)
print("DONE")

White to download: 49
Elaborating 9010 whith isbn 806527285
Error in bookKeeper id: 9010
Elaborating 1991039 whith isbn 8203180728
Error in bookKeeper id: 1991039
Elaborating 3503947 whith isbn 9773513912
Error in bookKeeper id: 3503947
Elaborating 3047849 whith isbn 61566128
Error in bookKeeper id: 3047849
Elaborating 3047850 whith isbn 006156608X
Error in bookKeeper id: 3047850
Elaborating 3313418 whith isbn 1416599398
Error in bookKeeper id: 3313418
Elaborating 2804776 whith isbn 1400139027
Error in bookKeeper id: 2804776
Elaborating 605015 whith isbn 7203116
Error in bookKeeper id: 605015
Elaborating 23965 whith isbn 8433972618
Error in bookKeeper id: 23965
Elaborating 6224895 whith isbn 9791680000
Error in bookKeeper id: 6224895
Elaborating 13425570 whith isbn 62102427
Error in bookKeeper id: 13425570
Elaborating 251964 whith isbn 60727217
Error in bookKeeper id: 251964
Elaborating 8765461 whith isbn 1441213570
Error in bookKeeper id: 8765461
Elaborating 11041 whith isbn 157912574

In [70]:
nowhite = alldata[alldata.color_name != "White"]
nowhite
white = alldata[alldata.color_name == "White"]
white

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,price(USD),color_name
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,49.78,White
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,...,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...,7.52,White
12,13,5470,5470,153313,995,451524934,9.780452e+12,"George Orwell, Erich Fromm, Celâl Üster",1949.0,Nineteen Eighty-Four,...,45518,41845,86425,324874,692021,908229,https://images.gr-assets.com/books/1348990566m...,https://images.gr-assets.com/books/1348990566s...,9.99,White
21,22,12232938,12232938,1145090,183,316166685,9.780316e+12,Alice Sebold,2002.0,The Lovely Bones,...,36642,62777,131188,404699,583575,479323,https://images.gr-assets.com/books/1457810586m...,https://images.gr-assets.com/books/1457810586s...,15.97,White
32,33,930,929,1558965,220,739326228,9.780739e+12,Arthur Golden,1997.0,Memoirs of a Geisha,...,25605,23500,59033,258700,517157,559782,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,2.99,White
39,40,19501,19501,3352398,185,143038419,9.780143e+12,Elizabeth Gilbert,2006.0,"Eat, pray, love: one woman's search for everyt...",...,49714,100373,149549,310212,332191,314272,https://images.gr-assets.com/books/1503066414m...,https://images.gr-assets.com/books/1503066414s...,7.85,White
41,42,1934,1934,3244642,1707,451529308,9.780452e+12,Louisa May Alcott,1868.0,Little Women,...,17090,31645,70011,250794,426280,535563,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,0.25,White
49,50,30119,30119,30518,45,60513039,9.780061e+12,Shel Silverstein,1974.0,Where the Sidewalk Ends: The Poems and Drawing...,...,9234,16590,30792,139024,293222,544153,https://images.gr-assets.com/books/1168052448m...,https://images.gr-assets.com/books/1168052448s...,3.30,White
53,54,11,386162,3078186,257,345391802,9.780345e+12,Douglas Adams,1979.0,The Hitchhiker's Guide to the Galaxy,...,20345,21764,41962,145173,299579,498001,https://images.gr-assets.com/books/1327656754m...,https://images.gr-assets.com/books/1327656754s...,7.19,White
54,55,5129,5129,3204877,515,60929871,9.780061e+12,Aldous Huxley,1932.0,Brave New World,...,20095,26367,60328,219895,389379,383166,https://images.gr-assets.com/books/1487389574m...,https://images.gr-assets.com/books/1487389574s...,13.95,White


In [71]:
white = white.drop(axis = 1, columns=["color_name"])
colorlist = []
for index, row in white.iterrows():
    print("Elbarating id: "+str(row["id"])+" bookid: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
    try:
        col = CoverColorPredict(str(row["book_id"]))
        colorpredicted = col.predictColor()
        colorlist.append(colorpredicted[1])
    except:
        print("Error in color predict id: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
        colorlist.append(None)
        coloridlist.append(None)

newwhite = white.assign(color_name = colorlist)
newalldata = pd.concat([nowhite, newwhite], ignore_index =True)

Elbarating id: 2 bookid: 3 isbn10 439554934
Elbarating id: 10 bookid: 1885 isbn10 679783261
Elbarating id: 13 bookid: 5470 isbn10 451524934
Elbarating id: 22 bookid: 12232938 isbn10 316166685
Elbarating id: 33 bookid: 930 isbn10 739326228
Elbarating id: 40 bookid: 19501 isbn10 143038419
Elbarating id: 42 bookid: 1934 isbn10 451529308
Elbarating id: 50 bookid: 30119 isbn10 60513039
Elbarating id: 54 bookid: 11 isbn10 345391802
Elbarating id: 55 bookid: 5129 isbn10 60929871
Elbarating id: 57 bookid: 37435 isbn10 142001740
Elbarating id: 59 bookid: 24178 isbn10 64410935
Elbarating id: 63 bookid: 6185 isbn10 393978893
Elbarating id: 66 bookid: 18405 isbn10 446675539
Elbarating id: 72 bookid: 11588 isbn10 450040186
Elbarating id: 75 bookid: 227443 isbn10 014028009X
Elbarating id: 78 bookid: 5139 isbn10 307275558
Elbarating id: 81 bookid: 7445 isbn10 074324754X
Elbarating id: 82 bookid: 1845 isbn10 385486804
Elbarating id: 84 bookid: 7677 isbn10 030734813X
Elbarating id: 86 bookid: 32542 isb

Elbarating id: 1301 bookid: 1110 isbn10 385340540
Elbarating id: 1302 bookid: 8235178 isbn10 1595143971
Elbarating id: 1309 bookid: 9439303 isbn10 297854399
Elbarating id: 1310 bookid: 85301 isbn10 676979343
Elbarating id: 1313 bookid: 42389 isbn10 743464117
Elbarating id: 1314 bookid: 22463 isbn10 785819118
Elbarating id: 1315 bookid: 25179 isbn10 1891830430
Elbarating id: 1318 bookid: 51504 isbn10 876850867
Elbarating id: 1319 bookid: 469571 isbn10 679744398
Elbarating id: 1323 bookid: 769483 isbn10 60577312
Elbarating id: 1324 bookid: 781110 isbn10 689848919
Elbarating id: 1325 bookid: 1914973 isbn10 670018864
Elbarating id: 1330 bookid: 170548 isbn10 073380084X
Elbarating id: 1335 bookid: 60510 isbn10 778324338
Elbarating id: 1337 bookid: 64222 isbn10 60502932
Elbarating id: 1343 bookid: 34506 isbn10 61020702
Elbarating id: 1344 bookid: 6516450 isbn10 316078573
Elbarating id: 1346 bookid: 944076 isbn10 575077905
Elbarating id: 1349 bookid: 817791 isbn10 393330478
Elbarating id: 135

Elbarating id: 1646 bookid: 9534 isbn10 812565959
Elbarating id: 1647 bookid: 7704143 isbn10 nan
Error in color predict id: 7704143 isbn10 nan
Elbarating id: 1650 bookid: 37677 isbn10 1587263890
Elbarating id: 1655 bookid: 17204 isbn10 399154094
Elbarating id: 1656 bookid: 38743 isbn10 312330863
Elbarating id: 1661 bookid: 6905012 isbn10 61922315
Elbarating id: 1662 bookid: 6693 isbn10 375822070
Elbarating id: 1663 bookid: 68591 isbn10 9504915248
Elbarating id: 1665 bookid: 1268479 isbn10 nan
Error in color predict id: 1268479 isbn10 nan
Elbarating id: 1666 bookid: 42038 isbn10 1903436575
Elbarating id: 1667 bookid: 8492856 isbn10 670012947
Elbarating id: 1670 bookid: 104189 isbn10 849900417
Elbarating id: 1672 bookid: 10235 isbn10 812973011
Elbarating id: 1674 bookid: 9579634 isbn10 7423292
Elbarating id: 1675 bookid: 5187 isbn10 385265700
Elbarating id: 1676 bookid: 9593 isbn10 385333870
Elbarating id: 1677 bookid: 7661 isbn10 60872985
Elbarating id: 1680 bookid: 114159 isbn10 749935

Elbarating id: 1976 bookid: 25 isbn10 076790382X
Elbarating id: 1977 bookid: 65112 isbn10 64410161
Elbarating id: 1982 bookid: 17934530 isbn10 374104093
Elbarating id: 1983 bookid: 11479285 isbn10 nan
Elbarating id: 1986 bookid: 5345 isbn10 385517238
Elbarating id: 1989 bookid: 761732 isbn10 812551494
Elbarating id: 1990 bookid: 11331421 isbn10 1590514637
Elbarating id: 1992 bookid: 759945 isbn10 671646788
Elbarating id: 1993 bookid: 31332 isbn10 345434803
Elbarating id: 1998 bookid: 833253 isbn10 1565075722
Elbarating id: 2000 bookid: 3086160 isbn10 60393491
Elbarating id: 2004 bookid: 5306 isbn10 142000701
Elbarating id: 2006 bookid: 2890090 isbn10 553804693
Elbarating id: 2009 bookid: 14996 isbn10 553804804
Elbarating id: 2011 bookid: 1840511 isbn10 385517882
Elbarating id: 2012 bookid: 13140 isbn10 446692654
Elbarating id: 2013 bookid: 5015 isbn10 1400076196
Elbarating id: 2014 bookid: 185911 isbn10 345466608
Elbarating id: 2017 bookid: 4588949 isbn10 385735294
Elbarating id: 2020 

Elbarating id: 2310 bookid: 78427 isbn10 785289089
Elbarating id: 2311 bookid: 61549 isbn10 316058432
Elbarating id: 2312 bookid: 234724 isbn10 765340054
Elbarating id: 2314 bookid: 10979 isbn10 679732268
Elbarating id: 2315 bookid: 24335 isbn10 64408671
Elbarating id: 2316 bookid: 13641208 isbn10 812993802
Elbarating id: 2317 bookid: 6944032 isbn10 312593554
Elbarating id: 2318 bookid: 998 isbn10 671015206
Elbarating id: 2319 bookid: 5267365 isbn10 1423116380
Elbarating id: 2320 bookid: 14318 isbn10 743244583
Elbarating id: 2321 bookid: 26 isbn10 60920084
Elbarating id: 2322 bookid: 9500416 isbn10 140006712X
Elbarating id: 2323 bookid: 43326 isbn10 446607207
Elbarating id: 2324 bookid: 8248 isbn10 60885416
Elbarating id: 2325 bookid: 18630 isbn10 61053562
Elbarating id: 2326 bookid: 24384 isbn10 440228891
Elbarating id: 2328 bookid: 23940 isbn10 142003166
Elbarating id: 2329 bookid: 313605 isbn10 140157352
Elbarating id: 2331 bookid: 25111004 isbn10 1501110349
Elbarating id: 2332 book

...

Elbarating id: 8176 bookid: 8745 isbn10 688172377
Elbarating id: 8179 bookid: 4076 isbn10 1861977166
Elbarating id: 8180 bookid: 17899392 isbn10 316239992
Elbarating id: 8181 bookid: 107029 isbn10 1401204457
Elbarating id: 8183 bookid: 7879278 isbn10 1434764338
Elbarating id: 8184 bookid: 20890479 isbn10 385682824
Elbarating id: 8186 bookid: 116053 isbn10 340752459
Elbarating id: 8189 bookid: 9654 isbn10 451209435
Elbarating id: 8190 bookid: 9349915 isbn10 316176044
Elbarating id: 8191 bookid: 4765642 isbn10 9381626715
Elbarating id: 8192 bookid: 6694 isbn10 140568190
Elbarating id: 8193 bookid: 17934521 isbn10 1250039819
Elbarating id: 8195 bookid: 106882 isbn10 375435484
Elbarating id: 8196 bookid: 289601 isbn10 061858532X
Elbarating id: 8197 bookid: 493455 isbn10 1416936408
Elbarating id: 8199 bookid: 78971 isbn10 312939779
Elbarating id: 8211 bookid: 207687 isbn10 60525134
Elbarating id: 8212 bookid: 330760 isbn10 802131867
Elbarating id: 8213 bookid: 181400 isbn10 745917437
Elbara

Elbarating id: 8490 bookid: 797094 isbn10 345459210
Elbarating id: 8491 bookid: 92846 isbn10 345397673
Elbarating id: 8492 bookid: 676 isbn10 375755195
Elbarating id: 8494 bookid: 17970255 isbn10 nan
Elbarating id: 8496 bookid: 384457 isbn10 312363044
Elbarating id: 8498 bookid: 7992363 isbn10 2246760011
Elbarating id: 8500 bookid: 17455815 isbn10 nan
Elbarating id: 8505 bookid: 10844125 isbn10 031609739X
Elbarating id: 8506 bookid: 18392 isbn10 441010288
Elbarating id: 8510 bookid: 103815 isbn10 689857977
Elbarating id: 8514 bookid: 43352 isbn10 671888587
Elbarating id: 8516 bookid: 12349 isbn10 140280464
Elbarating id: 8517 bookid: 60925 isbn10 446696161
Elbarating id: 8518 bookid: 11665750 isbn10 nan
Elbarating id: 8520 bookid: 18045473 isbn10 nan
Elbarating id: 8522 bookid: 428862 isbn10 039953346X
Elbarating id: 8523 bookid: 70561 isbn10 60007761
Elbarating id: 8524 bookid: 270032 isbn10 874778883
Elbarating id: 8529 bookid: 36064 isbn10 037572737X
Elbarating id: 8530 bookid: 7124

Elbarating id: 8816 bookid: 4609710 isbn10 039915583X
Elbarating id: 8824 bookid: 37298 isbn10 425152898
Elbarating id: 8828 bookid: 13167847 isbn10 805095837
Elbarating id: 8829 bookid: 228194 isbn10 451190521
Elbarating id: 8830 bookid: 43345 isbn10 743484355
Elbarating id: 8832 bookid: 32636 isbn10 60596384
Elbarating id: 8833 bookid: 789559 isbn10 899190359
Elbarating id: 8835 bookid: 27423283 isbn10 nan
Error in color predict id: 27423283 isbn10 nan
Elbarating id: 8836 bookid: 6452806 isbn10 446539252
Elbarating id: 8837 bookid: 3760 isbn10 142002054
Elbarating id: 8838 bookid: 2978 isbn10 60594527
Elbarating id: 8840 bookid: 349929 isbn10 142437271
Elbarating id: 8841 bookid: 47701 isbn10 61238961
Elbarating id: 8842 bookid: 23232 isbn10 375836578
Elbarating id: 8843 bookid: 428223 isbn10 345495802
Elbarating id: 8844 bookid: 7507908 isbn10 1595143378
Elbarating id: 8845 bookid: 6713071 isbn10 1439172951
Elbarating id: 8846 bookid: 8701960 isbn10 375423729
Elbarating id: 8848 boo

Elbarating id: 9140 bookid: 1541884 isbn10 553806610
Elbarating id: 9141 bookid: 9329354 isbn10 575097361
Elbarating id: 9142 bookid: 6517407 isbn10 1439150001
Elbarating id: 9143 bookid: 7252 isbn10 60539097
Elbarating id: 9144 bookid: 85321 isbn10 446606243
Elbarating id: 9145 bookid: 17333237 isbn10 316098868
Elbarating id: 9146 bookid: 7261549 isbn10 7325940
Elbarating id: 9147 bookid: 228198 isbn10 451190548
Elbarating id: 9148 bookid: 111306 isbn10 821780654
Elbarating id: 9150 bookid: 17192373 isbn10 857520660
Elbarating id: 9152 bookid: 531197 isbn10 375828192
Elbarating id: 9155 bookid: 227590 isbn10 553576798
Elbarating id: 9156 bookid: 765193 isbn10 803710402
Elbarating id: 9158 bookid: 44543 isbn10 156007479
Elbarating id: 9159 bookid: 60220 isbn10 380790076
Elbarating id: 9162 bookid: 37377 isbn10 312346069
Elbarating id: 9163 bookid: 11104030 isbn10 345524942
Elbarating id: 9164 bookid: 20886354 isbn10 nan
Elbarating id: 9167 bookid: 22600903 isbn10 nan
Elbarating id: 916

Elbarating id: 9431 bookid: 550940 isbn10 399154388
Elbarating id: 9432 bookid: 74034 isbn10 014303653X
Elbarating id: 9435 bookid: 1153709 isbn10 670913502
Elbarating id: 9440 bookid: 92371 isbn10 451210468
Elbarating id: 9442 bookid: 134958 isbn10 60959517
Elbarating id: 9443 bookid: 571508 isbn10 1573451991
Elbarating id: 9446 bookid: 10049436 isbn10 7345631
Elbarating id: 9449 bookid: 125958 isbn10 1595542779
Elbarating id: 9452 bookid: 36058 isbn10 99366614
Elbarating id: 9453 bookid: 71787 isbn10 143035088
Elbarating id: 9454 bookid: 5052 isbn10 451216725
Elbarating id: 9455 bookid: 429024 isbn10 140437428
Elbarating id: 9456 bookid: 73860 isbn10 345339258
Elbarating id: 9457 bookid: 46908 isbn10 1595324097
Elbarating id: 9458 bookid: 17277800 isbn10 785165630
Elbarating id: 9462 bookid: 11445880 isbn10 1608196259
Elbarating id: 9465 bookid: 105920 isbn10 078510786X
Elbarating id: 9467 bookid: 20646604 isbn10 nan
Error in color predict id: 20646604 isbn10 nan
Elbarating id: 9468 

In [72]:
newalldata.to_csv("dataset\\finals\\allbooks.csv")